# Test Pyspark

In [11]:
import os
os.environ["SPARK_HOME"]='/Users/rhyando/.pyenv/versions/3.11.9/lib/python3.11/site-packages/pyspark'
# os.environ["PYSPARK_PYTHON"]='/Users/rhyando/.pyenv/versions/3.11.9/bin/python'
#os.environ["SPARK_LOCAL_IP"]='127.0.0.1'

In [12]:
#!pyenv local 3.12.2
#!pip install pyspark==3.5.3        a

In [13]:
import pyspark
from pyspark.sql import SparkSession

conf = pyspark.SparkConf().setAll([
    ('spark.app.name', 'test')
    , ('spark.master', "spark://spark-master:7077")
    #, ('spark.driver.host', 'localhost')
    #, ('spark.driver.bindAddress', 'localhost')
    ])
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=test, master=spark://spark-master:7077) created by __init__ at /var/folders/pq/lqzg8gbs1w1921px2hpkmxym0000gn/T/ipykernel_71537/4278044439.py:30 

In [4]:
!echo $SPARK_HOME

/Users/rhyando/.pyenv/versions/3.11.9/lib/python3.11/site-packages/pyspark


In [5]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder.appName("demo").getOrCreate()

In [4]:
df = spark.createDataFrame(
    [
        ("sue", 32),
        ("li", 3),
        ("bob", 75),
        ("heo", 13),
    ],
    ["first_name", "age"],
)


In [5]:
df.show()

+----------+---+
|first_name|age|
+----------+---+
|       sue| 32|
|        li|  3|
|       bob| 75|
|       heo| 13|
+----------+---+



# Test Unity Local

In [1]:
import os
os.environ["SPARK_HOME"]='/Users/rhyando/.pyenv/versions/3.11.9/lib/python3.11/site-packages/pyspark'
# os.environ["PYSPARK_PYTHON"]='/Users/rhyando/.pyenv/versions/3.11.9/bin/python'
#os.environ["SPARK_LOCAL_IP"]='127.0.0.1'

In [2]:
import pyspark
from pyspark.sql import SparkSession
import os
import socket

conf = pyspark.SparkConf().setAll([
    ('spark.driver.host', socket.gethostbyname(socket.gethostname()))
    , ('spark.app.name', 'test')
    , ('spark.master', "spark://spark-master:7077")
    , ('spark.driver.host', "host.docker.internal")
    # , ('spark.driver.bindAddress', "localhost")
    # , ("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.2.1,io.unitycatalog:unitycatalog-spark_2.12:0.2.0")
    , ("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    # , ('spark.hadoop.fs.s3a.endpoint', "http://minio-service.minio-dev.svc.cluster.local:6544")
    # # for minio spark
    # , ('spark.hadoop.fs.s3a.access.key','minio')
    # , ('spark.hadoop.fs.s3a.secret.key', 'minio123')
    # , ('spark.hadoop.fs.s3a.path.style.access', 'true')
    # , ("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # for iceberg minio jdbc
    , ("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    , ('spark.sql.catalog.spark_catalog', 'io.unitycatalog.spark.UCSingleCatalog')
    , ('spark.sql.catalog.unity', 'io.unitycatalog.spark.UCSingleCatalog')
    , ("spark.sql.catalog.unity.catalog-impl", "io.delta.sql.DeltaSparkSessionExtension")
    , ('spark.sql.catalog.unity.uri', 'http://127.0.0.1:4566')
    , ('spark.sql.catalog.unity.token', '')
    , ('spark.sql.defaultCatalog', 'unity')
    # , ('spark.sql.catalog.nessie_catalog.warehouse', 's3a://unity/')
    ])
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

25/01/26 17:28:14 WARN Utils: Your hostname, Rhyandos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.50.80 instead (on interface en0)
25/01/26 17:28:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/26 17:28:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("SHOW SCHEMAS in unity").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [4]:
spark.sql("SHOW TABLES IN unity.default").show()  

+---------+-----------------+-----------+
|namespace|        tableName|isTemporary|
+---------+-----------------+-----------+
|  default|        marksheet|      false|
|  default|marksheet_uniform|      false|
|  default|        mytable11|      false|
|  default|          numbers|      false|
|  default|   user_countries|      false|
+---------+-----------------+-----------+



In [5]:
spark.sql("SELECT * FROM unity.default.numbers").show()

AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `default`.`numbers` doesn't exist.

In [6]:
spark.sql('drop table demo.mytable')

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `unity`.`demo`.`mytable` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.

In [7]:
spark.sql("DROP SCHEMA demo")

AnalysisException: [SCHEMA_NOT_FOUND] The schema `demo` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a catalog, verify the current_schema() output, or qualify the name with the correct catalog.
To tolerate the error on drop use DROP SCHEMA IF EXISTS.

In [8]:
spark.sql("CREATE SCHEMA demo")

DataFrame[]

In [9]:
spark.sql("SHOW SCHEMAS").show()

+---------+
|namespace|
+---------+
|  default|
|     demo|
+---------+



In [10]:
spark.sql("""
CREATE TABLE demo.mytable (id INT, desc STRING)
USING delta
LOCATION '/tmp/fileshare/mytable'
""")

DataFrame[]

In [11]:
spark.sql("SELECT * FROM unity.demo.mytable").show()

25/01/26 17:28:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+------+
| id|  desc|
+---+------+
|  4|test 4|
|  1|test 1|
|  3|test 3|
|  2|test 2|
|  4|test 4|
|  4|test 4|
|  1|test 1|
|  2|test 2|
|  2|test 2|
|  3|test 3|
|  2|test 2|
|  3|test 3|
|  1|test 1|
|  3|test 3|
|  4|test 4|
|  1|test 1|
+---+------+



In [12]:
# Insert new rows
spark.sql("INSERT INTO demo.mytable VALUES (1, 'test 1')")
spark.sql("INSERT INTO demo.mytable VALUES (2, 'test 2')")
spark.sql("INSERT INTO demo.mytable VALUES (3, 'test 3')")
spark.sql("INSERT INTO demo.mytable VALUES (4, 'test 4')")

DataFrame[]

In [13]:
# Read table
spark.sql("SELECT * FROM demo.mytable").show()

+---+------+
| id|  desc|
+---+------+
|  4|test 4|
|  4|test 4|
|  1|test 1|
|  3|test 3|
|  3|test 3|
|  2|test 2|
|  4|test 4|
|  1|test 1|
|  4|test 4|
|  1|test 1|
|  2|test 2|
|  2|test 2|
|  3|test 3|
|  2|test 2|
|  3|test 3|
|  1|test 1|
|  3|test 3|
|  4|test 4|
|  2|test 2|
|  1|test 1|
+---+------+



In [26]:
# Read table
df = spark.sql("SELECT * FROM demo.mytable")
print(df.explain(mode='formatted'))

== Physical Plan ==
* ColumnarToRow (2)
+- Scan parquet unity.demo.mytable (1)


(1) Scan parquet unity.demo.mytable
Output [2]: [id#3132, desc#3133]
Batched: true
Location: PreparedDeltaFileIndex [file:/tmp/fileshare/mytable]
ReadSchema: struct<id:int,desc:string>

(2) ColumnarToRow [codegen id : 1]
Input [2]: [id#3132, desc#3133]


None


In [14]:
!curl -d "client_id=minio" \
     -d "client_secret=pnyxAvinHSWoccatyV3GtvTYBN6pkmnv" \
     -d "grant_type=password" \
     -d "username=admin" \
     -d "password=admin" \
     http://localhost:8080/realms/master/protocol/openid-connect/token

{"access_token":"eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJEd0JTbzVlWmR0d29YRERVdFRrTm1BX3FaT0lMemNWXzBqSHd3ZkFnYmtJIn0.eyJleHAiOjE3Mzc5MTI1NjYsImlhdCI6MTczNzkwODk2NiwianRpIjoiNmU2YWRhMjAtNzExZS00NjIwLTkyMDUtYzQ2NjBmODI4NWM0IiwiaXNzIjoiaHR0cDovL2xvY2FsaG9zdDo4MDgwL3JlYWxtcy9tYXN0ZXIiLCJhdWQiOlsibXlyZWFsbS1yZWFsbSIsIm1hc3Rlci1yZWFsbSIsImFjY291bnQiXSwic3ViIjoiZTE4NDdjMWEtMWEzOC00MTY4LTk5YjgtOGE4MDBjZDhmMTU0IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoibWluaW8iLCJzaWQiOiI2NDBkYThlZS05YzgzLTRkNTItYTVlYy0yNGQyMTM0NGRkOWUiLCJhY3IiOiIxIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImNyZWF0ZS1yZWFsbSIsImRlZmF1bHQtcm9sZXMtbWFzdGVyIiwib2ZmbGluZV9hY2Nlc3MiLCJhZG1pbiIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsibXlyZWFsbS1yZWFsbSI6eyJyb2xlcyI6WyJ2aWV3LXJlYWxtIiwidmlldy1pZGVudGl0eS1wcm92aWRlcnMiLCJtYW5hZ2UtaWRlbnRpdHktcHJvdmlkZXJzIiwiaW1wZXJzb25hdGlvbiIsImNyZWF0ZS1jbGllbnQiLCJtYW5hZ2UtdXNlcnMiLCJxdWVyeS1yZWFsbXMiLCJ2aWV3LWF1dGhvcml6YXRpb24iLCJxdWVyeS1jbGllbnRzIiwicXVlcnktdXNlcnMiLCJtYW5hZ2UtZXZlbnRzIiw

In [ ]:
MINIO_IDENTITY_OPENID_CLIENT_SECRET=RwY1B5yQ1VuQFcuMf18HkVhjMRn6MVw1
pnyxAvinHSWoccatyV3GtvTYBN6pkmnv

In [1]:
import os
os.environ["SPARK_HOME"]='/Users/rhyando/.pyenv/versions/3.11.9/lib/python3.11/site-packages/pyspark'
# os.environ["PYSPARK_PYTHON"]='/Users/rhyando/.pyenv/versions/3.11.9/bin/python'
#os.environ["SPARK_LOCAL_IP"]='127.0.0.1'

In [2]:
import pyspark
from pyspark.sql import SparkSession
import os
import socket

conf = pyspark.SparkConf().setAll([
    ('spark.driver.host', socket.gethostbyname(socket.gethostname()))
    , ('spark.app.name', 'test')
    , ('spark.master', "spark://spark-master:7077")
    , ('spark.driver.host', "host.docker.internal")
    # , ('spark.driver.bindAddress', "localhost")
    # , ("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-spark_2.12:3.2.1,io.unitycatalog:unitycatalog-spark_2.12:0.2.0")
    , ("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    , ('spark.hadoop.fs.s3a.endpoint', "http://minio-service.minio-dev.svc.cluster.local:6544")
    # for minio spark
    , ('spark.hadoop.fs.s3a.access.key','minio')
    , ('spark.hadoop.fs.s3a.secret.key', 'minio123')
    , ('spark.hadoop.fs.s3a.path.style.access', 'true')
    , ("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # for iceberg minio jdbc
    , ("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    , ('spark.sql.catalog.spark_catalog', 'io.unitycatalog.spark.UCSingleCatalog')
    , ('spark.sql.catalog.unity', 'io.unitycatalog.spark.UCSingleCatalog')
    , ("spark.sql.catalog.unity.catalog-impl", "io.delta.sql.DeltaSparkSessionExtension")
    , ('spark.sql.catalog.unity.uri', 'http://127.0.0.1:4566')
    , ('spark.sql.catalog.unity.token', '')
    , ('spark.sql.defaultCatalog', 'unity')
    # , ('spark.sql.catalog.nessie_catalog.warehouse', 's3a://unity/')
    ])
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.config(conf=conf).getOrCreate()

25/01/26 17:50:28 WARN Utils: Your hostname, Rhyandos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.50.80 instead (on interface en0)
25/01/26 17:50:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/26 17:50:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sql("SELECT * FROM unity.demo.mytable").show()

25/01/26 17:50:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+------+
| id|  desc|
+---+------+
|  4|test 4|
|  4|test 4|
|  1|test 1|
|  3|test 3|
|  3|test 3|
|  2|test 2|
|  4|test 4|
|  1|test 1|
|  4|test 4|
|  1|test 1|
|  2|test 2|
|  2|test 2|
|  3|test 3|
|  2|test 2|
|  3|test 3|
|  1|test 1|
|  3|test 3|
|  4|test 4|
|  2|test 2|
|  1|test 1|
+---+------+



In [ ]:
spark.sql("""
CREATE TABLE demo.mytable112 (id INT, desc STRING)
USING delta
LOCATION 's3a://unity/mytable112'
""")

25/01/26 17:50:48 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/01/26 19:14:03 WARN TaskSetManager: Lost task 0.0 in stage 12.0 (TID 108) (172.19.0.4 executor 0): org.apache.hadoop.fs.s3a.AWSClientIOException: getFileStatus on s3a://unity/mytable112/_delta_log/00000000000000000000.json: com.amazonaws.SdkClientException: Unable to execute HTTP request: minio-service.minio-dev.svc.cluster.local: Try again: Unable to execute HTTP request: minio-service.minio-dev.svc.cluster.local: Try again
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:214)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:175)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3799)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.extractOrFetchSimpleFileStatus(S3AFileSystem.java:5401)
	at

In [1]:
!ls /Users/rhyando/.pyenv/versions/3.11.9/lib/python3.11/site-packages/pyspark/jars/

HikariCP-2.5.1.jar
JLargeArrays-1.5.jar
JTransforms-3.1.jar
RoaringBitmap-0.9.45.jar
ST4-4.0.4.jar
activation-1.1.1.jar
aircompressor-0.27.jar
algebra_2.12-2.0.1.jar
annotations-17.0.0.jar
antlr-runtime-3.5.2.jar
antlr4-4.9.3.jar
antlr4-runtime-4.9.3.jar
aopalliance-repackaged-2.6.1.jar
arpack-3.0.3.jar
arpack_combined_all-0.1.jar
arrow-format-12.0.1.jar
arrow-memory-core-12.0.1.jar
arrow-memory-netty-12.0.1.jar
arrow-vector-12.0.1.jar
audience-annotations-0.5.0.jar
avro-1.11.2.jar
avro-ipc-1.11.2.jar
avro-mapred-1.11.2.jar
blas-3.0.3.jar
bonecp-0.8.0.RELEASE.jar
breeze-macros_2.12-2.1.0.jar
breeze_2.12-2.1.0.jar
cats-kernel_2.12-2.1.1.jar
chill-java-0.10.0.jar
chill_2.12-0.10.0.jar
commons-cli-1.5.0.jar
commons-codec-1.16.1.jar
commons-collections-3.2.2.jar
commons-collections4-4.4.jar
commons-compiler-3.1.9.jar
commons-compress-1.23.0.jar
commons-crypto-1.1.0.jar
commons-dbcp-1.4.jar
commons-io-2.16.1.jar
commons-lang-2.6.jar
commons-lang3-3.12.0.jar
commons-logging-1.1.3.jar
commons

In [1]:
bin/uc table create --full_name unity.default.mytable11 --columns "col1 int, col2 double" --storage_location s3://unity/default/mytable11

SyntaxError: invalid syntax (3144423103.py, line 1)

In [ ]:
bin/uc table write --full_name unity.default.mytable11